# <div align="center" > The custom environment for the Project Portfolio simulation </div>
<div align="center" > This is the simulation notebook for the project portfolio environement simulation.
</br>
</br>


---
--- 
# The prerequisites
<span style="padding-left: 2em;">This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>


## Project management relative concepts
<span style="padding-left: 2em;">This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>

### Project Cost Performance Baseline
<img src="../Assets/Project-cost-baseline-graph.jpg" alt="drawing" width="400"/>


---
---
## Problem deffinition

<span style="padding-left: 2em;">This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.</span>

## Inclusions and exclusions


### environment design 

<span style="padding-left: 2em;">here's an infograph of environment and it's architecture</span>

<img src="../assets/Info_graphs/environment/Mindmap - 2025.10.30-Environment.webp" alt="drawing" width="800"/>

---
---
# 🚀Initiation

In [1]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Boxs
import numpy as np



ImportError: cannot import name 'Boxs' from 'gymnasium.spaces' (f:\Career\01. IE (Industrial Engineering)\00. Univesity\Masters\05. Master's thesis\PPO-project-portfolio-budgeting-AI-agent\.venv\Lib\site-packages\gymnasium\spaces\__init__.py)

In [ ]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

2.0.1+cu117
CUDA available: True
Device name: NVIDIA GeForce 940MX
